# ![](https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png) Project 4: Logistic Regression and Job Search Sites

![](jobhunting.jpg)

### Description

This week, we learned about classification models like the logistic regression. Now, we're going to put these skills to the test.

You're working as a data scientist for a contracting firm that's rapidly expanding. Now that they have their most valuable employee (you!), they need to leverage data to win more contracts. Your firm offers technology and scientific solutions and wants to be competitive in the hiring market. Your principal thinks the best way to gauge salary amounts is to take a look at what industry factors influence the pay scale for these professionals.

Aggregators like [Indeed.com](https://www.indeed.com) regularly pool job postings from a variety of markets and industries. Your job is to understand what factors most directly impact data science salaries and effectively, accurately find appropriate data science related jobs in your metro region.

#### Project Summary

In this project, we are going to collect salary information on data science jobs in a variety of markets. Then using the location, title, and summary of the job, we will attempt to predict a corresponding salary for that job. While most listings DO NOT come with salary information (as you will see in this exercise), being to able extrapolate or predict the expected salaries for other listings will be extremely useful for negotiations :)

Normally we could use regression for this task; however, instead we will convert this into a classification problem and use Logistic Regression.

- **Question**: Why would we want this to be a classification problem?
- **Answer**: While more precision may be better, there is a fair amount of natural variance in job salaries; therefore, predicting a range be may be useful.

The first part of assignment will be focused on scraping [Indeed.com](www.indeed.com) and the second will be focused on using the listings with salary information to build a model and predict salaries.

Your job is to:

1. Inspect data from [Indeed.com](www.indeed.com) on data science salary trends for your analysis.
  - Select and parse data from at least 1000 postings for jobs, potentially from multiple location searches.
2. Find out what factors most directly impact salaries (Title, location, department, etc.). In this case, we do not want to predict mean salary as would be done in a regression. Your boss believes that salary is better represented in categories than continuously
  - Test, validate, and describe your models. What factors predict salary category? How do your models perform?
3. Author a report to your Principal detailing your analysis.

**BONUS PROBLEMS:**

1. Your boss would rather tell a client incorrectly that they would get a lower salary job than tell a client incorrectly that they would get a high salary job. Adjust one of your logistic regression models to ease his mind, and explain what it is doing and any tradeoffs. Plot the ROC curve.
2. Text variables and regularization:

- **Part 1**: Job descriptions contain more potentially useful information you could leverage. Use the job summary to find words you think would be important and add them as predictors to a model.
- **Part 2**: Gridsearch parameters for Ridge and Lasso for this model and report the best model.


**Goal:** clean data, run logistic regression, derive insights, present findings.

---

### Necessary Deliverables / Submission

- Materials must be in a clearly labeled Jupyter notebook.
- Materials must be pushed to student's github master branch.
- Materials must be submitted by the end of Week 5.

---

### Dataset

1. We'll be utilizing a dataset derived from live web data: [Indeed.com](https://www.indeed.com)

---



### Project Feedback + Evaluation

[Attached here is a complete rubric for this project.](./project-04-rubric.md)


## Packages to be used. (The import areas)

In [63]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from itertools import chain
import re
from collections import defaultdict
import nltk
nltk.download("stopwords")
nltk.download("punkt")
import string
from operator import itemgetter


[nltk_data] Downloading package stopwords to /Users/chris/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/chris/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Import the data from the .csv.  Inspect, transform & clean whatever needs to be modification.


In [2]:
raw_jobs_df = pd.read_csv('../assets/indeed-scraped-job-postings.csv')
raw_jobs_df.shape

(413, 6)

In [3]:
raw_jobs_df.columns

Index([u'city', u'company', u'salary', u'summary', u'title', u'parsed_salary'], dtype='object')

In [4]:
raw_jobs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 413 entries, 0 to 412
Data columns (total 6 columns):
city             413 non-null object
company          413 non-null object
salary           413 non-null object
summary          413 non-null object
title            413 non-null object
parsed_salary    406 non-null float64
dtypes: float64(1), object(5)
memory usage: 19.4+ KB


In [5]:
# check for null values in city, parsed_salary, summary, title
print (raw_jobs_df['parsed_salary'].unique()) # check for 0 or unrealistic values. Found none
print "City Count: ", raw_jobs_df['city'].count()
print "Parsed_Salary Count: ", (raw_jobs_df['parsed_salary'].count())  # check for # of nan values.


[  87792.    85127.5   75000.    68983.5   27000.    93645.    66654.
   77500.   130000.    85000.    55733.5   43500.    44250.   105000.
   47500.   170000.   140000.   150000.    35000.   160000.    41000.
   65000.   120000.   135000.    82500.   190000.   125000.    40000.
   44293.5   81078.    71592.    87313.5       nan   59910.    80000.
  110000.    90000.    56000.    60000.   165000.   100000.    70000.
  115285.5   10000.   115000.   145000.    74782.5   79150.    42000.
   49832.5   45162.    26442.    39390.    42723.5   30264.    23322.
   60528.    50375.   107500.    32900.    95000.   155000.    71000.
  138300.   200000.   175000.    55000.   185000.    50502.5   50000.
  180000.   137500.    82329.    58000.   112500.    41600.    75500.
   87368.5   74317.5  132500.   122500.    85038.5  225000.    58948.5
  195000.    74903.5   76941.5   59250.    45488.   102500.    75065.
   99521.    66244.    75619.5   39754.5   45000.    61500.    87500.
  152500.    67500.

There are some nan values in the parsed salary field.  These will need to addressed.

The city field is complete.



In [6]:
city_info_df = raw_jobs_df.groupby(by = 'city', as_index = False).agg({'title':{'count':len},
    'parsed_salary': {"mean":np.mean, "std": np.std}})

In [7]:
city_info_df.columns = [' '.join(col).strip() for col in city_info_df.columns.values]

In [8]:
city_lambda = lambda x: 0 if x <= raw_jobs_df['parsed_salary'].mean() else 1
city_info_df['high_low'] = city_info_df['parsed_salary mean'].apply(city_lambda)

In [9]:
city_info_df

,city,parsed_salary std,parsed_salary mean,title count,high_low
0,Atlanta,46360.341439,85279.552632,19,0
1,Austin,52873.613907,96571.428571,7,0
2,Boston,36928.395538,116635.904255,47,1
3,Chicago,41315.253132,118959.750000,36,1
4,Dallas,33022.346658,97958.333333,12,0
5,Denver,34497.431749,78196.441176,17,0
6,Houston,19221.127177,64405.333333,9,0
7,Los+Angeles,50008.230234,112636.260870,24,1
8,Miami,29122.845056,76795.833333,6,0
9,New+York,53627.742423,107889.147059,103,1


Now, I'll add the high_salary field to the raw jobs data frame.

In [10]:
hi_low = lambda x: 0 if x <= raw_jobs_df['parsed_salary'].median() else 1
raw_jobs_df['high_salary'] = raw_jobs_df['parsed_salary'].apply(hi_low)

In [11]:
raw_jobs_df.head()

,city,company,salary,summary,title,parsed_salary,high_salary
0,Denver,Department Of The Interior,"$76,341 - $99,243 a year","Would you like to join the more than 10,000 sc...","Statistician, GS-1350-12 (DEU-PERM-DS)",87792.0,0
1,Denver,Department Of The Interior,"$71,012 - $99,243 a year",Investigate potential uses of geospatial data ...,Interdisciplinary Cartographer/Geographer - GS...,85127.5,0
2,Denver,Mental Health Center of Denver,"$70,000 - $80,000 a year",Advise the Data Developer with regard to creat...,Financial Data Scientist,75000.0,0
3,Denver,Denver Public Schools,"$62,712 - $75,255 a year",Portal managers on student outcome data report...,SENIOR RESEARCH ANALYST,68983.5,0
4,Denver,University of Colorado,"$25,000 - $29,000 a year",Experience entering and manipulating data in a...,Animal Care I,27000.0,0


Let's try a logistical regression on the cities.  I'll make dummy variables for the cities.  My guess is that the sign of coefficents will match the high city column in the city_info_df.

In [12]:
#model with label encoder.



encoder_model_df = raw_jobs_df.dropna()
le = LabelEncoder()
le.fit(encoder_model_df['city'])
le.classes_
X = le.transform(encoder_model_df['city']).reshape(-1,1)
y = encoder_model_df['high_salary'].reshape(-1,1)

# sklearn output
model = LogisticRegression()
model.fit(X,y)

# make predictions
expected = y
predicted = model.predict(X)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))




             precision    recall  f1-score   support

          0       0.56      0.75      0.64       208
          1       0.59      0.39      0.47       198

avg / total       0.58      0.57      0.56       406

[[155  53]
 [121  77]]


/Users/chris/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [13]:
#model with dummies
city_dummy_df = raw_jobs_df.dropna()
city_dummy_df = pd.get_dummies(data = city_dummy_df, columns=['city'])
y2 = encoder_model_df['high_salary'].reshape(-1,1)
X2 = city_dummy_df[['city_Atlanta', 'city_Austin', 'city_Boston',
       'city_Chicago', 'city_Dallas', 'city_Denver', 'city_Houston',
       'city_Los+Angeles', 'city_Miami', 'city_New+York', 'city_Palo+Alto',
       'city_Philadelphia', 'city_Phoenix', 'city_Pittsburgh',
       'city_Portland', 'city_San+Diego', 'city_San+Francisco',
       'city_Seattle']]
model2 = LogisticRegression()
model2.fit(X2,y2)
# make predictions
expected2 = y2
predicted2 = model2.predict(X2)
# summarize the fit of the model
print(metrics.classification_report(expected2, predicted2))
print(metrics.confusion_matrix(expected2, predicted2))

             precision    recall  f1-score   support

          0       0.69      0.65      0.67       208
          1       0.65      0.69      0.67       198

avg / total       0.67      0.67      0.67       406

[[136  72]
 [ 62 136]]


In [14]:
print "Accuracy with city names: %.3f" % model.score(X,y)
print "Accuracy with cities as dummy variables: %.3f " % model2.score(X2,y2)

Accuracy with city names: 0.571
Accuracy with cities as dummy variables: 0.670 


In [15]:
city_dummy_predict = pd.DataFrame(zip(X2.columns, np.transpose(model2.coef_)))
city_dummy_predict['mean_prediction'] = city_info_df['high_low']
city_dummy_predict

,0,1,mean_prediction
0,city_Atlanta,[-0.700448531384],0
1,city_Austin,[0.0784986158953],0
2,city_Boston,[1.06716003292],1
3,city_Chicago,[0.579601319232],1
4,city_Dallas,[-0.205064681095],0
5,city_Denver,[-0.824732795582],0
6,city_Houston,[-1.33534135632],0
7,city_Los+Angeles,[0.425132963613],1
8,city_Miami,[-0.599724295858],0
9,city_New+York,[0.0920758268619],1


Using the cities as dummy variables seems to provide slightly better results.  Also, the sign of the coefficient seems to match if the city average is above or below the national average.  Since the relationship of averages is easier for me to conceptualize and explain, I will most likely use that as a field in the future.

There does appear to be a strong relationship of cities to salary, and number of jobs found. This is a completely unsurprising result.  Based on regional variations of industries and cost of living, salaries in different cities should not be directly comparable.  I will adjust the job ratings of High/low to be based on the city average, not the national median.

In [16]:
city_dict_maker = city_info_df[['city', 'parsed_salary mean']]
city_dict_maker.set_index('city', inplace=True)
city_mean_dict = city_dict_maker.to_dict()['parsed_salary mean'] 



In [17]:
raw_jobs_df['city_mean']  = raw_jobs_df['city'].map(city_mean_dict)
raw_jobs_df['high_salary'] = np.where(raw_jobs_df['city_mean'] < raw_jobs_df['parsed_salary'],1,0)

In [18]:
print "Percentage of 'High salary jobs after adjusting for city variation: %.0f%%" % (raw_jobs_df['high_salary'].mean()*100)

Percentage of 'High salary jobs after adjusting for city variation: 47%


There is our new baseline accuracy.  47% of jobs as high salary jobs.

Now the additional features from the title and summary should be added to improve the prediction of High/Low salary jobs. Let's make a list of words in the title, and list of words in the summaries.  

If I'm feeling wacky, a dict of word counts might be a good idea, too. 

What the heck, let's map/reduce it because it is fun to use a blacksmith's hammer where a tack hammer is appropriate.  I'm starting with idea and code from "Data Science from Scratch", chapt 24.  

In [19]:
title_word_list = raw_jobs_df['title'].tolist()
title_word_count_dict = defaultdict(int)

for line in title_word_list:
     for word in re.findall(r'[\w]+', re.sub('[^A-z]', ' ',line)):
            title_word_count_dict[word.lower()] += 1 


From the title word dictionary, drop the stop words, and things with only more than 2 occurances.  That should yield words with some predictive power.

In [20]:
stops =  nltk.corpus.stopwords.words('english')
title_significant_dict = {}
for k,v in title_word_count_dict.items():
    #print k,v
    if (k not in stops) & (v > 1) :
        title_significant_dict[k] = v


There maybe an advantage to merging "sr" & "Senior" and other terms with similar meaning. Or setting the threshhold of the word count higher.  I'll investigate this later. Maybe.

Time to do this on the Summary. I expect this is be less than optimally effective because the summaries have been truncated. 

Just for giggles, I'll set the count level to 3 or more occurances.  And I'll run the Porter stemmer, because I can.
Stemming didn't significantly reduce the number of features.  The count went from 480 to 440.

Post script.  Unicode errors are happening.  Maybe encoding on reading needs to be changed....




In [21]:
summary_word_list = raw_jobs_df['summary'].tolist()
summary_word_count_dict = defaultdict(int)

port = nltk.stem.PorterStemmer()

i = 0
for line in summary_word_list:
    line_read = line.translate(None, string.punctuation)
    line_read = re.sub('[^[A-z\s]','',line_read)
    words = nltk.tokenize.word_tokenize(line_read)
    for word in words:
#        w = port.stem(word)
        summary_word_count_dict[word.lower()] += 1 

summary_significant_dict = {}
for k,v in summary_word_count_dict.items():
     if (k not in stops) & (v > 3) :
            summary_significant_dict[k] = v

I'm setting the threshold for Summary words to 4.  Later, I might reconsider a threshold of 2 or no threshold.

Let's take a different tack:  sklearn.feature_extraction.text.Count_vectorizer.  Let's see what happens... Starting with titles.



In [22]:
from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS

In [71]:
title_vocab = title_significant_dict.keys()
# title_vocab = ['administrator', 'analysis', 'analyst', 'analytics', 'architect', 'c']
cv = CountVectorizer(vocabulary=title_vocab)
X_title = cv.fit_transform(raw_jobs_df['title'])
y_title = raw_jobs_df['high_salary']

title_model = LogisticRegression(penalty='l1', verbose = 1, n_jobs =-1)
title_model.fit(X_title, y_title)
title_significant_words = [(w,B) for w,B in zip(title_vocab, np.transpose(title_model.coef_[0])) if B != 0]


[LibLinear]

Since this used the l1 penalty, the insignificant values were pushed to zero.  What is left is approx 50 significant words in the titles.  

Now for the Summaries:

In [25]:
summary_vocab = summary_significant_dict.keys()
cv = CountVectorizer(vocabulary=summary_vocab)
X_summary = cv.fit_transform(raw_jobs_df['title'])
y_summary = raw_jobs_df['high_salary']

summary_model = LogisticRegression(penalty='l1', verbose = 1, n_jobs =-1)
summary_model.fit(X_summary, y_summary)
summary_significant_words = [w for w,B in zip(summary_vocab, np.transpose(summary_model.coef_[0])) if B != 0]


[LibLinear]

In [26]:
print summary_significant_words


['program', 'risk', 'level', 'principal', 'machine', 'data', 'healthcare', 'operations', 'scientist', 'developer', 'python', 'financial', 'research', 'health', 'director', 'quality', 'engineer', 'management', 'quantitative', 'senior', 'analyst', 'equity', 'intelligence', 'analytics', 'project', 'learning', 'modeling', 'java', 'firm', 'vp', 'analysis', 'lead', 'clinical']


There is a list of about 30 significant words in the summary.  This doesn't seems to realistics.  There are not many skills or specialities listed.  I expect 2 factors are contributing to this. First, the summaries are truncated and the skills were likely to be dropped. Second, the sample size is fairly small.  Some likely candidates for important words like spark, hadoop or geospatial were not included.  My conjecture is that both factors contributed to excluding many important words.

Let's combine the models, and do a cross validation to check quality of the model.

In [30]:
combo_vocab = set(summary_vocab + title_vocab)

cv_combo = CountVectorizer(vocabulary=combo_vocab)
X_combo = cv.fit_transform(raw_jobs_df['title']+raw_jobs_df['summary'])
y_combo = raw_jobs_df['high_salary']

combo_model = LogisticRegression(penalty='l1', verbose = 1, n_jobs =-1)
combo_model.fit(X_combo, y_combo)
combo_significant_words = [(w,B) for w,B in zip(combo_vocab, np.transpose(combo_model.coef_[0])) if B != 0]

[LibLinear]

In [34]:
combo_significant_words

[('ability', 0.21758610714918436),
 ('accuracy', 0.030654017208986271),
 ('activities', -0.32092218701298342),
 ('administrator', 0.6246478740883975),
 ('analyst', 1.4576162543689895),
 ('analytical', -0.17930478005635139),
 ('analytics', 0.34038192853593913),
 ('asset', 0.09655162636949792),
 ('associate', -1.506127169260947),
 ('b', -0.0046310951672375355),
 ('candidate', 0.27660753625265494),
 ('care', 0.20785770148802427),
 ('collection', -0.078930588253961875),
 ('companys', 0.71849846956434293),
 ('consumer', -0.34760820939114823),
 ('control', -0.13988902766491759),
 ('create', 0.13407192501016127),
 ('currently', -0.22519022353403179),
 ('customer', 0.31698664017955075),
 ('deu', 0.060316463737283547),
 ('developer', 0.40723227217584995),
 ('development', 0.38883898245908161),
 ('disease', -0.95427799440925742),
 ('doctoral', 0.50137036229997867),
 ('electronics', 0.1242585940494694),
 ('engineers', 0.34331798379440182),
 ('etc', -0.074018062611025004),
 ('evaluate', -0.3725946

I'm not sure a combined model works better than separate models.  

In [72]:
title_significant_words.sort(key = itemgetter(1),reverse=True)
title_significant_words

[('quantitative', 2.7893223079600831),
 ('programmer', 2.3091111571759027),
 ('equity', 2.1946274938298407),
 ('python', 1.2137505880622053),
 ('director', 1.137825536184385),
 ('lead', 1.1255256321084735),
 ('operations', 1.0402671458983834),
 ('principal', 0.85938864140000082),
 ('developer', 0.7879799702927619),
 ('scientist', 0.74984171764995278),
 ('senior', 0.74703611545899218),
 ('statistician', 0.64798966919080525),
 ('engineer', 0.60836317090331382),
 ('vp', 0.60483418499298425),
 ('intelligence', 0.45185461172623026),
 ('analytics', 0.45166563349525357),
 ('machine', 0.44904384773369743),
 ('start', 0.43684535039396383),
 ('data', 0.38315136448253501),
 ('end', 0.35104591745021346),
 ('front', 0.3225049466285454),
 ('program', 0.30624151797342886),
 ('learning', 0.19555357271197377),
 ('modeling', 0.15006935829993023),
 ('marketing', 0.10709072780412049),
 ('interdisciplinary', 0.091542512960558889),
 ('hedge', 0.040230176162212042),
 ('science', 0.037969860758806456),
 ('sr'

## Report to principal:

The location is dominate for predicting the salary.  After adjusting for location, the most positively influential words in the title are: quantitative, programmer, equity, director, lead and operations.  The most negativly influential words are: contract, level, management, analyst, quality and "II". 



To Dos:  Cross validate models.  compare performances, perhaps combine the models but keep the title words separate from summary words.

Web scrape more data.  400 records isn't a satisfactory sample size.

Use all words from the title and summary, except stop words.

Plot the ROC Curve.

tweak a model to favor False Negatives over False Positives.

Expand report to principal.


